In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import utils
import plot_utils

# Read data

In [10]:
data_file = './data/paleo_meso_ceno_data_paleogeography_epoch.csv'
data = pd.read_csv(data_file)
data = data.sort_values(by=['mean_ma'], ascending=False)

time_file = './data/timeScale.xlsx'
time = pd.read_excel(time_file,sheet_name='Sheet1')
time_period = utils.get_timesubset(time,3,541)
width_period = [time_period['max_ma'][i] - time_period['min_ma'][i] for i in list(time_period.index)]

# Skeletal abundance

In [11]:
data_skeletal = utils.get_stats(data,'skeletal_total','epoch')
data_skeletal = data_skeletal.sort_values(by=['time'], ascending=True)

# Text analysis

In [12]:
abstracts = pd.read_excel('./data/marine_limestone_abstracts.xlsx','Sheet2')

mean_ma = [float(time_period[dat.Period.capitalize() == time_period.interval_name].iloc[0].mean_ma) for _,dat in abstracts.iterrows()]
abstracts['mean_ma'] = mean_ma
abstracts = abstracts.sort_values(by=['mean_ma'], ascending=False)

# Platform vs ramps

In [13]:
data_ramp_skeletal = utils.get_stats(data[data.framework=='ramp'],'skeletal_total','epoch')
data_platform_skeletal = utils.get_stats(data[data.framework=='platform'],'skeletal_total','epoch')

data_ramp_skeletal = data_ramp_skeletal.sort_values(by=['time'], ascending=True)
data_platform_skeletal = data_platform_skeletal.sort_values(by=['time'], ascending=True)

# Paleo geography

In [14]:
data_skeletal_tropics = utils.get_stats(data[data.paleo_geography=='Tropics'],'skeletal_total','epoch')
data_skeletal_extratropics = utils.get_stats(data[data.paleo_geography=='Extra-tropics'],'skeletal_total','epoch')

data_skeletal_tropics = data_skeletal_tropics.sort_values(by=['time'], ascending=True)
data_skeletal_extratropics = data_skeletal_extratropics.sort_values(by=['time'], ascending=True)


# Water depth

In [15]:
water_depth = []
for _,dat in data.iterrows():
    if (dat.depo_env1=='distalRamp') or (dat.depo_env1=='outerShelf'):
        water_depth.append('Deep')
    else:
        water_depth.append('Shallow')
        
data['water_depth'] = water_depth

data_skeletal_shallow = utils.get_stats(data[data.water_depth=='Shallow'],'skeletal_total','epoch')
data_skeletal_deep = utils.get_stats(data[data.water_depth=='Deep'],'skeletal_total','epoch')

data_skeletal_shallow = data_skeletal_shallow.sort_values(by=['time'], ascending=True)
data_skeletal_deep = data_skeletal_deep.sort_values(by=['time'], ascending=True)


# Plot Figure 1

In [16]:
#colorspecs
shell_bed_color = '#993d3d'
ramp_color = '#3500ca'
platform_color = '#ffb000'
abstract_color = '#fe8173'

fig = make_subplots(rows=6, cols=2, column_widths=[0.9,0.1],
                    row_heights = [0.19,0.19,0.19,0.19,0.19,0.05],
                    specs=[[{"secondary_y": False},{"secondary_y": False}],
                           [{ "secondary_y": True}, {"secondary_y": False}],
                           [{"secondary_y": False}, {"secondary_y": False}],
                           [{"secondary_y": False}, {"secondary_y": False}],
                           [{"secondary_y": False}, {"secondary_y": False}],
                           [{"secondary_y": False}, {"secondary_y": False}],
                          ],
                    shared_yaxes = True,shared_xaxes = True,vertical_spacing = 0.001,horizontal_spacing=0.008)

#Row 1
#Raw data
fig.add_trace(go.Scatter(x = data[data['mean_ma']>=1.4]['mean_ma'],y = data[data['mean_ma']>=1.4]['skeletal_total'],mode = 'markers',
                         name = 'All data', showlegend = True, legendgroup='1',
                         marker = dict(color = 'grey',size = 5,opacity=0.6),
                         ),row=1,col=1)

fig.add_trace(go.Scatter(x = data_skeletal['time'][2:],y = data_skeletal['mean'][2:],mode = 'lines+markers',
                         name = 'Mean', legendgroup='1',
                         showlegend = True,
                         marker = dict(color = 'black',size = 8)
                         ),row=1,col=1)
fig.update_xaxes(range=(540,-2),ticks = 'outside',showline = True, showticklabels = False, mirror = True,
                          linecolor = 'black',row=1,col=1)

fig.add_trace(go.Scatter(x = data[data['mean_ma']<=1.4]['mean_ma'],y = data[data['mean_ma']<=1.4]['skeletal_total'],mode = 'markers',
                         showlegend = False,
                         marker = dict(color = 'grey',size = 5,opacity=0.6),
                         ),row=1,col=2)
fig.add_trace(go.Scatter(x = data_skeletal['time'][:2],y = data_skeletal['mean'][:2],mode = 'lines+markers',
                         showlegend = False,
                         marker = dict(color = 'black',size = 8)
                         ),row=1,col=2)

# x_axes
fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=1,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False, mirror = True,
                          linecolor = 'black',row=1,col=2)

# y_axes
fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=1,col=1)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=1,col=2);

#Row 2
#Skeletal abundance & text analysis
plot_utils.plot_skeletal_grain(fig,data_skeletal[2:],'Skeletal biomass',2,1,True,legendgroup='2')
plot_utils.plot_skeletal_grain(fig,data_skeletal[:2],'Skeletal biomass',2,2,False)


fig.add_trace(go.Scatter(y=abstracts['ratio'],x=abstracts['mean_ma'],
                     marker_color=abstract_color, mode = 'lines+markers',
                    showlegend=False),secondary_y=True,
                          row=2,col=1)

# x axes
fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=2,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False, mirror = True,
                          linecolor = 'black',row=2,col=2);

fig.update_yaxes(ticks = 'outside',showline = True,
                         mirror = True,linecolor = 'black',row=2,col=1)

fig.update_yaxes(ticks = 'outside',showline = True,
                         mirror = True,linecolor = 'black',row=2,col=2);

fig.add_vline(x=443.4,line=dict(dash='dot',color='grey'),
              annotation=dict(text='end-Ordovician',textangle=270,yshift=70),
              annotation_position='left',row=2,col=1)
fig.add_vline(x=371.5,line=dict(dash='dot',color='grey'),
              annotation=dict(text='late-Devonian',textangle=270,yshift=70),
              annotation_position='left',row=2,col=1)
fig.add_vline(x=252.17,line=dict(dash='dot',color='grey'),
              annotation=dict(text='end-Permian',textangle=270,yshift=70),
              annotation_position='right',row=2,col=1)

fig.add_vline(x=201.3,line=dict(dash='dot',color='grey'),
              annotation=dict(text='end-Triassic',textangle=270,yshift=70),
              annotation_position='right',row=2,col=1)

fig.add_vline(x=66,line=dict(dash='dot',color='grey'),
              annotation=dict(text='end-Cretaceous',textangle=270,yshift=-70),
              annotation_position='right',row=2,col=1);

#row 3
#Platforms vs ramps

fig.add_trace(go.Scatter(x = data_ramp_skeletal[data_ramp_skeletal['time']>1.5]['time'],y = data_ramp_skeletal[data_ramp_skeletal['time']>1.5]['mean'],
                         mode = 'lines+markers',
                         name = 'Ramp', line = dict(color = ramp_color,dash='dash'),showlegend = True,
                         legendgroup='3',
                         marker = dict(color = ramp_color,size = 8),
                        ),row=3,col=1)

fig.add_trace(go.Scatter(x = data_platform_skeletal[data_platform_skeletal['time']>1.5]['time'],y = data_platform_skeletal[data_platform_skeletal['time']>1.5]['mean'],
                         mode = 'lines+markers',
                         name = 'Platform', line = dict(color = platform_color),showlegend = True,
                         legendgroup='3',
                         marker = dict(color = platform_color,size = 8),
                        ),row=3,col=1)


fig.add_trace(go.Scatter(x = data_ramp_skeletal[data_ramp_skeletal['time']<=1.5]['time'],y = data_ramp_skeletal[data_ramp_skeletal['time']<=1.5]['mean'],
                         mode = 'lines+markers',
                         name = 'Ramp', line = dict(color = ramp_color,dash='dash'),showlegend = False,
                         legendgroup='3',
                         marker = dict(color = ramp_color,size = 8),
                        ),row=3,col=2)

fig.add_trace(go.Scatter(x = data_platform_skeletal[data_platform_skeletal['time']<=1.5]['time'],y = data_platform_skeletal[data_platform_skeletal['time']<=1.5]['mean'],
                         mode = 'lines+markers',
                         name = 'Platform', line = dict(color = platform_color),showlegend = False,
                         marker = dict(color = platform_color,size = 8),
                        ),row=3,col=2)

fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=3,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=3,col=2);

fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True, title_font_family='Helvetica',
                         mirror = True,linecolor = 'black',row=3,col=1)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=3,col=2);

#Row 4
#paleogeography

fig.add_trace(go.Scatter(x = data_skeletal_tropics[data_skeletal_tropics['time']>1.5]['time'],
                         y = data_skeletal_tropics[data_skeletal_tropics['time']>1.5]['mean'],mode = 'lines+markers',
                             name = 'Tropics', showlegend = True,legendgroup='4',
                             marker = dict(color = 'red',size = 8,symbol='diamond'),
                             ),row=4,col=1)
fig.add_trace(go.Scatter(x = data_skeletal_tropics[data_skeletal_tropics['time']<=1.5]['time'],
                         y = data_skeletal_tropics[data_skeletal_tropics['time']<=1.5]['mean'],mode = 'lines+markers',
                             name = 'Tropics', showlegend = False,
                             marker = dict(color = 'red',size = 8,symbol='diamond'),
                             ),row=4,col=2)


fig.add_trace(go.Scatter(x = data_skeletal_extratropics[(data_skeletal_extratropics['time']<380) & (data_skeletal_extratropics['time']>1.5)]['time'],
                         y = data_skeletal_extratropics[(data_skeletal_extratropics['time']<380) & (data_skeletal_extratropics['time']>1.5)]['mean'],
                         mode = 'lines+markers',
                             name = 'Extratropics', showlegend = True,legendgroup='4',
                             marker = dict(color = 'blue',size = 8,symbol='triangle-up'),
                         line = dict(dash='dash'),
                             ),row=4,col=1)

fig.add_trace(go.Scatter(x = data_skeletal_extratropics[data_skeletal_extratropics['time']<=1.5]['time'],
                         y = data_skeletal_extratropics[data_skeletal_extratropics['time']<=1.5]['mean'],
                         mode = 'lines+markers',
                             name = 'Extratropics', showlegend = False,
                             marker = dict(color = 'blue',size = 8,symbol='triangle-up'),
                         line = dict(dash='dash'),
                             ),row=4,col=2)

fig.add_trace(go.Scatter(x = data_skeletal_extratropics[data_skeletal_extratropics['time']>=380]['time'],
                         y = data_skeletal_extratropics[data_skeletal_extratropics['time']>=380]['mean'],
                         mode = 'lines+markers',
                             name = 'Extra tropics', showlegend = False,legendgroup='4',
                             marker = dict(color = 'blue',size = 8,symbol='triangle-up'),
                         line = dict(dash='dash'),
                             ),row=4,col=1)


fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=4,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=4,col=2);

fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True, title_font_family='Helvetica',
                         mirror = True,linecolor = 'black',row=4,col=1)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=4,col=2);

#row 5
#water depth
fig.add_trace(go.Scatter(x = data_skeletal_shallow[data_skeletal_shallow['time']>1.5]['time'],
                         y = data_skeletal_shallow[data_skeletal_shallow['time']>1.5]['mean'],mode = 'lines+markers',
                             name = 'Shallow', showlegend = True,legendgroup='5',
                             marker = dict(color = '#A0E4F1',size = 8,symbol='circle'),
                         
                             ),row=5,col=1)

fig.add_trace(go.Scatter(x = data_skeletal_deep[data_skeletal_deep['time']>1.5]['time'],
                         y = data_skeletal_deep[data_skeletal_deep['time']>1.5]['mean'],mode = 'lines+markers',
                             name = 'Deep', showlegend = True,legendgroup='5',
                             marker = dict(color = '#03256C',size = 8,symbol='star-diamond'),
                         line = dict(dash='dash'),
                             ),row=5,col=1)

fig.add_trace(go.Scatter(x = data_skeletal_shallow[data_skeletal_shallow['time']<=1.5]['time'],
                         y = data_skeletal_shallow[data_skeletal_shallow['time']<=1.5]['mean'],mode = 'lines+markers',
                             name = 'Shallow', showlegend = False,
                             marker = dict(color = '#A0E4F1',size = 8,symbol='circle'),
                         
                             ),row=5,col=2)

fig.add_trace(go.Scatter(x = data_skeletal_deep[data_skeletal_deep['time']<=1.5]['time'],
                         y = data_skeletal_deep[data_skeletal_deep['time']<=1.5]['mean'],mode = 'lines+markers',
                             name = 'Deep', showlegend = False,
                             marker = dict(color = '#03256C',size = 8,symbol='star-diamond'),
                         line = dict(dash='dash'),
                             ),row=5,col=2)

fig.update_xaxes(range=(540,2.01),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=5,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',showline = True, showticklabels = False,mirror = True,
                          linecolor = 'black',row=5,col=2)

fig.update_yaxes(title = 'Skeletal biomass %',ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=5,col=1)
fig.update_yaxes(ticks = 'outside',showline = True,title_font_family='Helvetica', 
                         mirror = True,linecolor = 'black',row=5,col=2);

#Row 6
#time
plot_utils.plot_time(fig,time_period[1:],width_period[1:],6,1)
fig.add_trace(go.Bar(x = time_period['min_ma'][:1], y = time_period['scale_no']*0.2,showlegend = False,
                    offset=0,marker_color = time_period['color'],width=2.5,
                    text = time_period.interval_name,textposition = 'inside',
                     x0 = 0,dx = 100,
                    textfont = dict(size = 12,color = 'black', family='Arial black')),
                  row=6,col=2)
    
fig.update_yaxes(showticklabels = False,row=6,col=2)

#x_axes
fig.update_xaxes(range=(540,2.01),ticks = 'outside',title = 'Time (Ma)',row=6,col=1)
fig.update_xaxes(range=(2.5,-0.1),ticks = 'outside',title = 'Time (Ma)',row=6,col=2)


fig.update_layout(height = 1600, width = 1100,font_color = 'black', font_family='Helvetica',font_size=14,
                 paper_bgcolor = 'white',
                 plot_bgcolor = 'white',
                  xaxis1=dict(domain=[0.07, 0.8388]),
                  # xaxis3=dict(domain=[0.07, 0.8388]),
                  xaxis3=dict(domain=[0.07, 0.8388]),
                  xaxis5=dict(domain=[0.07, 0.8388]),
                  xaxis7=dict(domain=[0.07, 0.8388]),
                  xaxis9=dict(domain=[0.07, 0.8388]),
                  xaxis11=dict(domain=[0.07, 0.8388]),

                  yaxis1=dict(range=(0,105)),
                  yaxis3=dict(range=(0,85),),
                  # yaxis4=dict(title='',tickcolor=abstract_color),
                  # yaxis5=dict(title=''),
                  yaxis4=dict(title='log(skeletal/non-skeletal) abstracts',title_font_family='Helvetica',
                              overlaying='y3',side='left',anchor='free',position=0,linecolor=abstract_color,
                             titlefont=dict(color=abstract_color),tickfont=dict(color=abstract_color),tickcolor=abstract_color),
                  yaxis6=dict(range=(0,85)),
                  yaxis8=dict(range=(0,85)),
                  yaxis10=dict(range=(0,85)),
                  yaxis12=dict(range=(0,0.2)),
                 legend=dict(bgcolor='rgba(0,0,0,0)',valign='top',
                yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=0.08),
                  legend_tracegroupgap = 230,
                 )


fig.show()
fig.write_image('./figures/Figure_1.png',scale = 6)